<a href="https://colab.research.google.com/github/Aniket1492/PdfML/blob/main/UsingGeminiAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import google.generativeai as genai

In [24]:
# Used to securely store your API key
from google.colab import userdata

In [25]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)


In [26]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)



models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [27]:
# Model Configuration
MODEL_CONFIG = {
  "temperature": 0.1,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

## Safety Settings of Model
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

In [28]:
model = genai.GenerativeModel(model_name = "gemini-pro-vision",
                              generation_config = MODEL_CONFIG,
                              safety_settings = safety_settings)

In [29]:
from pathlib import Path

def image_format(image_path):
    img = Path(image_path)

    if not img.exists():
        raise FileNotFoundError(f"Could not find image: {img}")

    image_parts = [
        {
            "mime_type": "image/png", ## Mime type are PNG - image/png. JPEG - image/jpeg. WEBP - image/webp
            "data": img.read_bytes()
        }
    ]
    return image_parts



In [30]:
def gemini_output(image_path, system_prompt, user_prompt):

    image_info = image_format(image_path)
    input_prompt= [system_prompt, image_info[0], user_prompt]
    response = model.generate_content(input_prompt)
    return response.text

In [36]:
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               """

image_path = "/content/sample_data/AmazonTest1.PNG"

user_prompt = "What is the Total in the image?"

gemini_output(image_path, system_prompt, user_prompt)

' The total is 175.'

In [38]:
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               """
#system_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image "
image_path = "/content/sample_data/AmazonTest1.PNG"
user_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image "


In [39]:
output = gemini_output(image_path, system_prompt, user_prompt)

In [40]:
from IPython.display import Markdown
Markdown(output)

 ```json
{
  "Invoice": {
    "Supplier": {
      "Name": "ETRADEMARKETING PRIVATE LIMITED",
      "Address": {
        "Line1": "Building No. 801, ESR Pune Estates Pvt Ltd,",
        "Line2": "Village Ambethan, Tal. Khed, Dist. Pune, Pin",
        "City": "CHAKAN",
        "State": "MAHARASHTRA",
        "Pin": "410501",
        "Country": "IN"
      },
      "GSTIN": "27AADCV4254H1Z8"
    },
    "Recipient": {
      "Name": "Aniket Chaudhari",
      "Address": {
        "Line1": "SR NO 282/3/1/1 Porwal Road, Kanekar Nagar,",
        "Line2": "Lohgaon. Near Apple Chemist, PUNE",
        "City": "Pune",
        "State": "MAHARASHTRA",
        "Pin": "411047",
        "Country": "IN"
      }
    },
    "InvoiceNumber": "MH-NPQ-1317922175-2324",
    "InvoiceDate": "2023-04-02",
    "PlaceOfSupply": "MAHARASHTRA",
    "Items": [
      {
        "Description": "BLOOMBUDDY Organic Growth Booster Fertilizer Manure for Home Garden plants (1 KG), Granules, Pack of 1",
        "HSNCode": "B072V2C572",
        "UnitPrice": 368.58,
        "Discount": -18.45,
        "Quantity": 1,
        "TaxRate": 2.5,
        "TaxType": "CGST",
        "Amount": 8.75
      },
      {
        "Description": "200gm-50 Seeds Tomato Vegetable Seeds (Red Pack of 2) BLOOMBUDDY B09LM8PQR (B09LM8PRQT)",
        "HSNCode": "120091",
        "UnitPrice": 100.00,
        "Discount": -35.00,
        "Quantity": 3,
        "TaxRate": 0.00,
        "TaxType": "IGST",
        "Amount": 0.00
      }
    ],
    "TotalAmount": 175.50
  }
}
```